In [ ]:
# Model Class 

import pandas as pd 
import numpy as np 
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import pickle 

class Model :
    def __init__(self):
        self.model = pickle.load(open('model.sav','rb')) 
        train_data = pd.read_csv('train_data.csv')
        self.X_train = train_data.drop('Diseases', axis = 1)
        self.y_train = train_data.loc[:,['Diseases']]    
        self.le = LabelEncoder()
        self.le.fit(self.y_train)

    def predict(self,selected_cols=[]):
        if (len(selected_cols)>0) : 
            n = len(self.X_train.columns)
            symptoms = self.X_train.columns
            X = pd.DataFrame(np.zeros((1,n)), columns = symptoms)
            for col in selected_cols : 
                X.loc[0,col] = 1 
            res = [[indx,prob] for indx,prob  in enumerate(self.model.predict_proba(X).reshape(-1)) ]
            res = sorted(res,key=lambda x : -x[1])
            res = res[:5]
            for i in range(5) : 
                res[i][0] = self.le.inverse_transform([res[i][0]])[0]
            print(res)

cl = Model()   

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [ ]:
# Data Class
 

import pandas as pd 
import numpy as np 

class Data() : 
    def __init__(self) :
        self.data = pd.read_csv('train_data.csv')
        self.symptoms_df = pd.read_csv('Symptoms.csv')
        self.code_sym = [col for col in self.data.columns if col!='Diseases']
        self.descriptions = { col : list( self.symptoms_df[ self.symptoms_df['Code'] == col ]['Description'] )[0] \
                                    for col in self.code_sym }
        freq_counts = { col : self.data[col].sum() for col in self.code_sym}
        freq_counts = {k: v for k,v in sorted(freq_counts.items(), key= lambda item : -item[1]) }
        freq_counts_keys = list(freq_counts.keys())
        
        self.prev_options = np.random.choice(freq_counts_keys[:20],10, replace=False)
        self.prev_options_des = [self.descriptions[col] for col in self.prev_options]
        self.cur_options = [{'label':label , 'value':value } for label,value in zip(self.prev_options_des,self.prev_options) ]
        self.rem_options = [col for col in self.code_sym ]
        # self.selected_so_far = [] 
        self.fig = dict({
                        "data": [{"type": "bar",
                        "x": [],
                        "y": []}],
                        "layout": {"title": {"text": "Disease vs their probabilities"}}
                    })
        self.female_symp = ['fm_'+str(i) for i in range(6) ]

    def update_options(self, selected_options):
        new_options = []

        # case-1 : No option selected 
        if (len(selected_options) == 0 ):
            self.rem_options = [col for col in self.rem_options if col not in self.prev_options]
            new_options = self.rem_options[:6]
            new_options.extend(self.rem_options[-1:-5:-1])
            new_options = list(set(new_options))
            self.prev_options = new_options 

        # case - 2 : At least 1 options selected
        elif(len(selected_options)>0):
            self.rem_options = [col for col in self.rem_options if col not in self.prev_options]
            tmp_data = self.data[ self.data[ selected_options ].sum(axis=1)>0 ][ self.rem_options ]
            tmp_cols_sum = tmp_data.sum(axis=0)
            tmp_cols_sum = tmp_cols_sum[tmp_cols_sum>0]
            tmp_cols_sum= tmp_cols_sum.sort_values(ascending=False)
            new_options = list(tmp_cols_sum.index[:6])
            new_options.extend(list(tmp_cols_sum.index[-1:-5:-1]))
            new_options = list(set(new_options))
            self.prev_options = new_options 

            del tmp_data, tmp_cols_sum 

        
        self.prev_options_des = [self.descriptions[col] for col in self.prev_options ]
        self.cur_options = [{'label':label,'value':value} for label,value in zip(self.prev_options_des,self.prev_options)]

        del new_options

        return self.cur_options

dl = Data() 

In [ ]:
# import libraries  dash, dcc, html , Input, Output 
import dash
import dash_core_components as dcc 
import dash_html_components as html
from dash.dependencies import Input, Output
# from Model_class_file_name import cl 
# from Data_class_file_name import dl

disp_options = dl.cur_options 

# initialise app 
App = dash.Dash(title = "Disease - Predictor")

styles = { 
            'background' : '#111111', 
            'text'   : '#7FDBFF',
            'font-size' : '50px'
        }

App.layout = html.Div(
    [
        html.H2('Disease - Predictor', style = {'textAlign': 'center', 'color': styles['text']} ),
        html.Hr(),
        html.Div(
            [
                html.Div(
                    [
                        html.H4('Gender:', style = {'display':'inline'} ),
                        dcc.RadioItems(id='gender',
                            options = [
                                {'label': 'Male', 'value' : 'M'},
                                {'label': 'Female', 'value' : 'F'},
                                {'label': 'Prefer not to say', 'value' : 'O'}
                            ],
                            style= {'display':'inline'}


                        ),
                        
                    ],

                ),

                html.Div(
                    [
                        html.H4('Age(5-99):'),
                        dcc.Input(type = 'number', min = 5, max = 99),
                    ], 
                    style = {'columnCount': 2, 'width': '50%'}
                ),
                
            ], style = {'columnCount': 2, 'margin':'15px'}
        ),
        html.Div(
            [
                html.H4('Select Symptoms :-'),
                dcc.Checklist(
                    options = disp_options ,
                    labelStyle = {'display': 'block', 'margin':'10px'},
                    id = 'disp-options'
                    value = []
                    
                    
                ),
            ], 
            style = {'height':'330px'}
        ),
        html.Hr() ,
        html.Button('Next',id='next-button',n_clicks=0, 
                    style={'height':'30px','width':'70px'}),
        dcc.Graph(id='bar-plot'),

       ],
)

@App.callback(
    Output(component_id = 'disp_options', component_property = 'options' ),
    Output(component_id = 'next-button', component_property = 'n_clicks'),
    Output(component_id = 'bar-plot', component_property = 'figure'),
    Input(component_id = 'disp_options', component_property = 'options'),
    Input(component_id = 'disp_options', component_property = 'value')
    Input(component_id = 'next-button', component_property = 'n_clicks'),
    Input(component_id = 'bar-plot', component_property = 'figure')
)

def callback_function_1(options, selected_options, n_clicks, figure):
    if(n_clicks):
        options = dl.update_options(selected_options)
        n_clicks = 0
        if(len(selected_options)>0):
            predictions = cl.predict(selected_options)
            predictions = list(predictions.loc[:,'Diseases'])
            probability = list(predictions.loc[:,'prob'])
            dl.fig['data'][0]['x'] = predictions
            dl.fig['data'][0]['y'] = probability

            figure = dl.fig
    return options, n_clicks, figure


@App.callback(
    Output(component_id='gender',component_property='value'),
    Input(component_id='gender',component_property='value')
)
def callback_function_2(gender) : 
    if (gender == None) : 
        dash.exceptions.PreventUpdate
    if (gender == 'M') :
        dl.rem_options = [col for col in dl.rem_options if col not in dl.female_symp ]
    return gender 

App.run_server(debug = True)




ModuleNotFoundError: No module named 'dash'

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a12a1b37-24ed-400a-a1a4-4803ffdccc47' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>